In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd

import pickle

# Usable Code

In [2]:
years = range(2008, 2020)
months = ['october','november','december','january','february','march','april','may','june']
#months2 = ['october','november','december']
#months3 = ['january','february','march','april','may','june']
game_link_ends = []
NoneType = type(None)
for year in years:
    if year == 2007:
        for month in months[:3]:
            url0 = 'https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year, month)
            response0 = requests.get(url0)
            page0 = response0.text
            page0 = page0.replace("<!--","").replace("-->","")
            soup0 = BeautifulSoup(page0)
            if type(soup0.find('table')) != NoneType:
                #Grab links to games where we can scrape our actual data from
                for link in soup0.find('table').find_all('td', attrs={"data-stat": "box_score_text"}):
                    game_link_ends.append(link.findNext().get('href'))
    #elif year == 2019:
    #    for month in months[3:]:
    #        url0 = 'https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year, month)
    #        response0 = requests.get(url0)
    #        page0 = response0.text
    #        page0 = page0.replace("<!--","").replace("-->","")
    #        soup0 = BeautifulSoup(page0)
    #        if type(soup0.find('table')) != NoneType:
    #            #Grab links to games where we can scrape our actual data from
    #           for link in soup0.find('table').find_all('td', attrs={"data-stat": "box_score_text"}):
    #                game_link_ends.append(link.findNext().get('href'))
    else:
        for month in months:
            url0 = 'https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year, month)
            response0 = requests.get(url0)
            page0 = response0.text
            page0 = page0.replace("<!--","").replace("-->","")
            soup0 = BeautifulSoup(page0)
            if type(soup0.find('table')) != NoneType:
                #Grab links to games where we can scrape our actual data from
                for link in soup0.find('table').find_all('td', attrs={"data-stat": "box_score_text"}):
                    game_link_ends.append(link.findNext().get('href'))

In [3]:
url1 = 'https://www.basketball-reference.com'
url_list = []
for game_link in game_link_ends:
    url = ''.join([url1, game_link])
    url_list.append(url)

In [4]:
print(game_link_ends)

['/boxscores/200710300SAS.html', '/boxscores/200710300GSW.html', '/boxscores/200710300LAL.html', '/boxscores/200710310IND.html', '/boxscores/200710310ORL.html', '/boxscores/200710310TOR.html', '/boxscores/200710310NJN.html', '/boxscores/200710310CLE.html', '/boxscores/200710310MEM.html', '/boxscores/200710310NOH.html', '/boxscores/200710310DEN.html', '/boxscores/200711010MIA.html', '/boxscores/200711010UTA.html', '/boxscores/200711010SEA.html', '/boxscores/200711020CHA.html', '/boxscores/200711020IND.html', '/boxscores/200711020ORL.html', '/boxscores/200711020ATL.html', '/boxscores/200711020CLE.html', '/boxscores/200711020NJN.html', '/boxscores/200711020BOS.html', '/boxscores/200711020MIN.html', '/boxscores/200711020NOH.html', '/boxscores/200711020CHI.html', '/boxscores/200711020SAS.html', '/boxscores/200711020LAC.html', '/boxscores/200711020PHO.html', '/boxscores/200711030PHI.html', '/boxscores/200711030WAS.html', '/boxscores/200711030MEM.html', '/boxscores/200711030DAL.html', '/boxsc

In [5]:
print(len(url_list),len(game_link_ends))

15521 15521


In [14]:
def game_stat_dict(url):
    '''
    From Basketball-Reference link stub, request gamepage, parse with BeautifulSoup, and
    collect 
        - game total (target) 
        - traditional stats (features)
        - efficiency stats (features)
    Return total & stats as a dictionary.
    ''' 
    game_dict={}
    
    game_dict['game_id'] = url[47:59]
    
    response = requests.get(url)
    page = response.text
    page = page.replace("<!--","").replace("-->","")
    soup = BeautifulSoup(page, 'html.parser')
    
    #scrape teams & total from scorebox at top of page
    teams = []
    data = []
    table = soup.find(id="div_line_score")
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        data.append([ele.text for ele in cols[-1]])

    headers = table_body.find_all('th')
    teams = ([ele.text for ele in headers])
    
    game_dict['total'] = sum([int(item) for sublist in data for item in sublist])
    game_dict['away'] = teams[0]
    game_dict['home'] = teams[1]
    
    data2 = []
    table = soup.find(id="div_four_factors")
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        data2.append([ele.text for ele in cols]) # Get rid of empty values

    game_dict['pace_v'] = data2[0][0]
    game_dict['pace_h'] = data2[1][0]
    game_dict['eFg_v'] = data2[0][1]
    game_dict['eFg_h'] = data2[1][1]
    game_dict['tov_v'] = data2[0][2]
    game_dict['tov_h'] = data2[1][2]
    game_dict['orb_v'] = data2[0][3]
    game_dict['orb_h'] = data2[1][3]
    game_dict['ft_fga_v'] = data2[0][4]
    game_dict['ft_fga_h'] = data2[1][4]
    game_dict['ortg_v'] = data2[0][5]
    game_dict['ortg_h'] = data2[1][5]
    
    #scrape traditional team stats (away team)
    data3 = []
    table = soup.find('table', id="box-{}-game-basic".format(game_dict['away']))
    footer = table.find('tfoot')
    cols = footer.find_all('td')
    data3.append([ele.text for ele in cols])

    game_dict['fg_v'] = data3[0][0]
    game_dict['fga_v'] = data3[0][1]
    game_dict['fg_per_v'] = data3[0][2]
    game_dict['threes_v'] = data3[0][3]
    game_dict['threes_att_v'] = data3[0][4]
    game_dict['threes_per_v'] = data3[0][5]
    game_dict['ft_v'] = data3[0][6]
    game_dict['fta_v'] = data3[0][7]
    game_dict['ft_per_v'] = data3[0][8]
    game_dict['drb_v'] = data3[0][10]
    game_dict['trb_v'] = data3[0][11]
    game_dict['ast_v'] = data3[0][12]
    game_dict['stl_v'] = data3[0][13]
    game_dict['blk_v'] = data3[0][14]
    game_dict['to_v'] = data3[0][15]
    game_dict['fouls_v'] = data3[0][16]
    
    #scrape advanced team stats (away team)
    data4 = []
    table = soup.find('table',id="box-{}-game-advanced".format(game_dict['away']))
    footer = table.find('tfoot')
    cols = footer.find_all('td')
    data4.append([ele.text for ele in cols])

    game_dict['ts_per_v'] = data4[0][1]
    game_dict['threes_ar_v'] = data4[0][3]
    game_dict['ft_ar_v'] = data4[0][4]
    game_dict['drb_per_v'] = data4[0][6]
    game_dict['trb_per_v'] = data4[0][7]
    game_dict['ast_per_v'] = data4[0][8]
    game_dict['stl_per_v'] = data4[0][9]
    game_dict['blk_per_v'] = data4[0][10]
    game_dict['user_per_v'] = data4[0][12]
    game_dict['drtg_v'] = data4[0][14]
    
    #scrape traditional team stats (home team)
    data5 = []
    table = soup.find('table', id="box-{}-game-basic".format(game_dict['home']))
    footer = table.find('tfoot')
    cols = footer.find_all('td')
    data5.append([ele.text for ele in cols])

    game_dict['fg_h'] = data3[0][0]
    game_dict['fga_h'] = data3[0][1]
    game_dict['fg_per_h'] = data3[0][2]
    game_dict['threes_h'] = data3[0][3]
    game_dict['threes_att_h'] = data3[0][4]
    game_dict['threes_per_h'] = data3[0][5]
    game_dict['ft_h'] = data3[0][6]
    game_dict['fta_h'] = data3[0][7]
    game_dict['ft_per_h'] = data3[0][8]
    game_dict['drb_h'] = data3[0][10]
    game_dict['trb_h'] = data3[0][11]
    game_dict['ast_h'] = data3[0][12]
    game_dict['stl_h'] = data3[0][13]
    game_dict['blk_h'] = data3[0][14]
    game_dict['to_h'] = data3[0][15]
    game_dict['fouls_h'] = data3[0][16]
    
    #scrape advanced team stats (home team)
    data6 = []
    table = soup.find('table',id="box-{}-game-advanced".format(game_dict['home']))
    footer = table.find('tfoot')
    cols = footer.find_all('td')
    data6.append([ele.text for ele in cols])

    game_dict['ts_per_h'] = data4[0][1]
    game_dict['threes_ar_h'] = data4[0][3]
    game_dict['ft_ar_h'] = data4[0][4]
    game_dict['drb_per_h'] = data4[0][6]
    game_dict['trb_per_h'] = data4[0][7]
    game_dict['ast_per_h'] = data4[0][8]
    game_dict['stl_per_h'] = data4[0][9]
    game_dict['blk_per_h'] = data4[0][10]
    game_dict['user_per_h'] = data4[0][12]
    game_dict['drtg_h'] = data4[0][14]

    return game_dict

In [15]:
game = game_stat_dict(url_list[1])
game

{'game_id': '200710300GSW',
 'total': 213,
 'away': 'UTA',
 'home': 'GSW',
 'pace_v': '105.1',
 'pace_h': '105.1',
 'eFg_v': '.483',
 'eFg_h': '.455',
 'tov_v': '15.2',
 'tov_h': '17.6',
 'orb_v': '34.8',
 'orb_h': '14.9',
 'ft_fga_v': '.333',
 'ft_fga_h': '.338',
 'ortg_v': '111.3',
 'ortg_h': '91.3',
 'fg_v': '240',
 'fga_v': '41',
 'fg_per_v': '90',
 'threes_v': '.456',
 'threes_att_v': '5',
 'threes_per_v': '11',
 'ft_v': '.455',
 'fta_v': '30',
 'ft_per_v': '36',
 'drb_v': '16',
 'trb_v': '40',
 'ast_v': '56',
 'stl_v': '24',
 'blk_v': '9',
 'to_v': '7',
 'fouls_v': '19',
 'ts_per_v': '.553',
 'threes_ar_v': '.122',
 'ft_ar_v': '.400',
 'drb_per_v': '85.1',
 'trb_per_v': '60.2',
 'ast_per_v': '58.5',
 'stl_per_v': '8.6',
 'blk_per_v': '13.0',
 'user_per_v': '100.0',
 'drtg_v': '91.3',
 'fg_h': '240',
 'fga_h': '41',
 'fg_per_h': '90',
 'threes_h': '.456',
 'threes_att_h': '5',
 'threes_per_h': '11',
 'ft_h': '.455',
 'fta_h': '30',
 'ft_per_h': '36',
 'drb_h': '16',
 'trb_h': '40'

In [16]:
season0708 = url_list[:1315]
len(season0708)
#season0708[1]

1315

In [17]:
url_list[1]

'https://www.basketball-reference.com/boxscores/200710300GSW.html'

In [18]:
def f (url):
    game_dict={}
    
    game_dict['game_id'] = url[47:59]
    
    response = requests.get(url)
    page = response.text
    page = page.replace("<!--","").replace("-->","")
    soup = BeautifulSoup(page, 'html.parser')
    
    #scrape teams & total from scorebox at top of page
    teams = []
    data = []
    table = soup.find(id="div_line_score")
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        data.append([ele.text for ele in cols[-1]])

    headers = table_body.find_all('th')
    teams = ([ele.text for ele in headers])
    
    game_dict['total'] = sum([int(item) for sublist in data for item in sublist])
    game_dict['away'] = teams[0]
    game_dict['home'] = teams[1]

    
    #scrape traditional team stats (away team)
    data3 = []
    table = soup.find('table', id="box-{}-game-basic".format(game_dict['away']))
    #footer = table.find('tfoot')
    #cols = footer.find_all('td')
    #data3.append([ele.text for ele in cols])

    '''
    game_dict['fg_v'] = data3[0][0]
    game_dict['fga_v'] = data3[0][1]
    game_dict['fg_per_v'] = data3[0][2]
    game_dict['threes_v'] = data3[0][3]
    game_dict['threes_att_v'] = data3[0][4]
    game_dict['threes_per_v'] = data3[0][5]
    game_dict['ft_v'] = data3[0][6]
    game_dict['fta_v'] = data3[0][7]
    game_dict['ft_per_v'] = data3[0][8]
    game_dict['drb_v'] = data3[0][10]
    game_dict['trb_v'] = data3[0][11]
    game_dict['ast_v'] = data3[0][12]
    game_dict['stl_v'] = data3[0][13]
    game_dict['blk_v'] = data3[0][14]
    game_dict['to_v'] = data3[0][15]
    game_dict['fouls_v'] = data3[0][16]
    '''
    return table

In [13]:
game_list = []

for url in season0708:
    game_list.append(f(url)) #scrape data
    print(f(url))

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-POR-game-basic">
<caption>Portland Trail Blazers (0-1) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goa

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-UTA-game-basic">
<caption>Utah Jazz (1-0) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal Attempts" s

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-HOU-game-basic">
<caption>Houston Rockets (1-0) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal Attem

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-WAS-game-basic">
<caption>Washington Wizards (0-1) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal At

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-MIL-game-basic">
<caption>Milwaukee Bucks (0-1) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal Attem

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-PHI-game-basic">
<caption>Philadelphia 76ers (0-1) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal At

<table class="sortable stats_table" data-cols-to-freeze=",1" id="box-CHI-game-basic">
<caption>Chicago Bulls (0-1) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header"><th></th>
<th aria-label="" class="over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
</tr>
<tr>
<th aria-label="Starters" class="poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
<th aria-label="Minutes Played" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
<th aria-label="Field Goals" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
<th aria-label="Field Goal Attempts" class="poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal Attempt

KeyboardInterrupt: 

In [20]:
game_list = []

for url in url_list:
    game_list.append(game_stat_dict(url)) #scrape data
    print(len(game_list))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


...

10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
1021

11413
11414
11415
11416
11417
11418
11419
11420
11421
11422
11423
11424
11425
11426
11427
11428
11429
11430
11431
11432
11433
11434
11435
11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
11448
11449
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
1157

12779
12780
12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
1294

14145
14146
14147
14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14176
14177
14178
14179
14180
14181
14182
14183
14184
14185
14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
1431

15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521


In [ ]:
game_list

In [22]:
game_df = pd.DataFrame(game_list) #convert list of dicts to DF
game_df #sanity check that we got all games and data

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.542,.297,.419,77.1,57.0,67.6,10.7,7.4,100.0,99.9
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.463,.202,.455,70.2,50.0,41.7,5.5,9.5,100.0,109.5
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.429,.195,.253,76.9,53.3,48.5,4.5,7.8,100.0,115.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.592,.415,.280,70.6,46.2,89.5,7.0,8.9,100.0,104.0
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.674,.463,.256,72.9,49.4,69.8,9.0,18.2,100.0,109.5
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.544,.372,.279,80.0,48.1,61.1,12.7,7.8,100.0,97.0
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.601,.512,.171,70.5,46.3,71.1,5.3,13.2,100.0,111.3


In [23]:
with open('game_df.pickle', 'wb') as to_write:
    pickle.dump(game_df, to_write)

# Scratch Work

Just checking to see if the process works, starting with March 2018 then the full 2018 season.

In [ ]:
#years = range(2011, 2019)
months = ['october','november','december','january','february','march','april','may','june']
years = [2018]
#months = ['march']
game_link_ends = []
for year in years:
    for month in months:
        url0 = 'https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year, month)
        response0 = requests.get(url0)
        page0 = response0.text
        page0 = page0.replace("<!--","").replace("-->","")
        soup0 = BeautifulSoup(page0)
        #Grab links to games where we can scrape our actual data from
        for link in soup0.find('table').find_all('td', attrs={"data-stat": "box_score_text"}):
            game_link_ends.append(link.findNext().get('href'))

In [ ]:
url1 = 'https://www.basketball-reference.com'
url_list = []
for game_link in game_link_ends:
    url = ''.join([url1, game_link])
    url_list.append(url)

In [ ]:
#Sanity check to make sure we get all the boxscore links & subsequent pages (222 games in March 2018)
print(len(url_list), len(game_link_ends))

### Workable Page

In [ ]:
url = url_list[1]
url

In [ ]:
def game_stat_dict(url):
    game_dict={}
    #headers = ['total','away','home']
    game_dict['game_id'] = url[47:59]
    
    response = requests.get(url)
    page = response.text
    page = page.replace("<!--","").replace("-->","")
    soup = BeautifulSoup(page, 'html.parser')
    
    #scrape totals from scorebox at top of page
    data = []
    table = soup.find(id="div_line_score")
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        #cols = [ele.text.strip() for ele in cols]
        data.append([ele.text for ele in cols[-1]]) # Get rid of empty values
    headers = table_body.find_all('th')
    teams = ([ele.text for ele in headers])
    
    game_dict['total'] = sum([int(item) for sublist in data for item in sublist])
    game_dict['away'] = teams[0]
    game_dict['home'] = teams[1]
    
    #game_dict = dict(zip(headers,[total,away,home]))
    return game_dict

In [ ]:
game = game_stat_dict(url)
game

In [ ]:
game_id = url[47:59]
game_id

In [ ]:
#Start with first game to get a sense of basic structure; REMOVE COMMENTS
response = requests.get(url)
page = response.text
page = page.replace("<!--","").replace("-->","")
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
soup

In [ ]:
#scrape totals from scorebox at top of page
data = []
table = soup.find(id="div_line_score")
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    #cols = [ele.text.strip() for ele in cols]
    data.append([ele.text for ele in cols[-1]]) # Get rid of empty values
headers = table_body.find_all('th')
teams = ([ele.text for ele in headers])
    
total = sum([int(item) for sublist in data for item in sublist])
away = teams[0]
home = teams[1]
print(total,home,away)

In [ ]:
#scrape "Four Factors" box
data2 = []
table = soup.find(id="div_four_factors")
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    #print(cols)
    #cols = [ele.text.strip() for ele in cols]
    data2.append([ele.text for ele in cols if ele]) # Get rid of empty values
    #teams.append([ele.text for ele in cols if ele])
data2

pace_v = data2[0][0]
pace_h = data2[1][0]
eFg_v = data2[0][1]
eFg_h = data2[1][1]
tov_v = data2[0][2]
tov_h = data2[1][2]
orb_v = data2[0][3]
orb_h = data2[1][3]
ft_fga_v = data2[0][4]
ft_fga_h = data2[1][4]
ortg_v = data2[0][5]
ortg_h = data2[1][5]
print(pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,orb_v,orb_h,ft_fga_v,ft_fga_h,ortg_v,ortg_h)

In [ ]:
#scrape traditional team stats (away team)
data3 = []
table = soup.find('table', id="box-{}-game-basic".format(away))
footer = table.find('tfoot')
cols = footer.find_all('td')
data3.append([ele.text for ele in cols])

fg_v = data3[0][0]
fga_v = data3[0][1]
fg_per_v = data3[0][2]
threes_v = data3[0][3]
threes_att_v = data3[0][4]
threes_per_v = data3[0][5]
ft_v = data3[0][6]
fta_v = data3[0][7]
ft_per_v = data3[0][8]
orb_v = data3[0][9]
drb_v = data3[0][10]
trb_v = data3[0][11]
ast_v = data3[0][12]
stl_v = data3[0][13]
blk_v = data3[0][14]
to_v = data3[0][15]
fouls_v = data3[0][16]
print(fg_v,fga_v,fg_per_v,threes_v,threes_att_v,threes_per_v,ft_v,
      fta_v,ft_per_v,orb_v,drb_v,trb_v,ast_v,stl_v,blk_v,to_v,fouls_v)

In [ ]:
table #away team traditional stats

In [ ]:
#scrape advanced team stats (away team)
data4 = []
table = soup.find('table',id="box-{}-game-advanced".format(away))
footer = table.find('tfoot')
cols = footer.find_all('td')
data4.append([ele.text for ele in cols])

ts_per_v = data4[0][1]
threes_ar_v = data4[0][3]
ft_ar_v = data4[0][4]
drb_per_v = data4[0][6]
trb_per_v = data4[0][7]
ast_per_v = data4[0][8]
stl_per_v = data4[0][9]
blk_per_v = data4[0][10]
user_per_v = data4[0][12]
drtg_v = data4[0][14]
print(ts_per_v,threes_ar_v,ft_ar_v,drb_per_v,trb_per_v,ast_per_v,stl_per_v,blk_per_v,user_per_v,drtg_v)

In [ ]:
#table #away team advanced stats

In [ ]:
#scrape traditional team stats (home team)
data5 = []
table = soup.find('table', id="box-{}-game-basic".format(home))
footer = table.find('tfoot')
cols = footer.find_all('td')
data5.append([ele.text for ele in cols])

fg_h = data3[0][0]
fga_h = data3[0][1]
fg_per_h = data3[0][2]
threes_h = data3[0][3]
threes_att_h = data3[0][4]
threes_per_h = data3[0][5]
ft_h = data3[0][6]
fta_h = data3[0][7]
ft_per_h = data3[0][8]
orb_h = data3[0][9]
drb_h = data3[0][10]
trb_h = data3[0][11]
ast_h = data3[0][12]
stl_h = data3[0][13]
blk_h = data3[0][14]
to_h = data3[0][15]
fouls_h = data3[0][16]
print(fg_h,fga_h,fg_per_h,threes_h,threes_att_h,threes_per_h,ft_h,
      fta_h,ft_per_h,orb_h,drb_h,trb_h,ast_h,stl_h,blk_h,to_h,fouls_h)

In [ ]:
#table #home team tradtional stats

In [ ]:
#scrape advanced team stats (home team)
data6 = []
table = soup.find('table',id="box-{}-game-advanced".format(home))
footer = table.find('tfoot')
cols = footer.find_all('td')
data6.append([ele.text for ele in cols])

ts_per_h = data4[0][1]
threes_ar_h = data4[0][3]
ft_ar_h = data4[0][4]
drb_per_h = data4[0][6]
trb_per_h = data4[0][7]
ast_per_h = data4[0][8]
stl_per_h = data4[0][9]
blk_per_h = data4[0][10]
user_per_h = data4[0][12]
drtg_h = data4[0][14]
print(ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h)

In [ ]:
#table #home team advanced stats

In [ ]:
headers = ['game_id','total','away','home','pace_v','pace_h','eFg_v','eFg_h','tov_v','tov_h',
                                  'orb_v','orb_h','ft_fga_v','ft_fga_h','ortg_v','ortg_h','fg_v','fga_v',
                                  'fg_per_v','threes_v','threes_att_v','threes_per_v','ft_v','fta_v',
                                  'ft_per_v','orb_v','drb_v','trb_v','ast_v','stl_v','blk_v','to_v','fouls_v',
                                  'ts_per_v','threes_ar_v','ft_ar_v','drb_per_v','trb_per_v','ast_per_v',
                                  'stl_per_v','blk_per_v','user_per_v','drtg_v','fg_h','fga_h','fg_per_h',
                                  'threes_h','threes_att_h','threes_per_h','ft_h','fta_h','ft_per_h',
                                  'orb_h','drb_h','trb_h','ast_h','stl_h','blk_h','to_h','fouls_h','ts_per_h',
                                  'threes_ar_h','ft_ar_h','drb_per_h','trb_per_h','ast_per_h','stl_per_v',
                                  'blk_per_v','user_per_v','drtg_v']

In [ ]:
features = [game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,
                                  orb_v,orb_h,ft_fga_v,ft_fga_h,ortg_v,ortg_h,fg_v,fga_v,
                                  fg_per_v,threes_v,threes_att_v,threes_per_v,ft_v,fta_v,
                                  ft_per_v,orb_v,drb_v,trb_v,ast_v,stl_v,blk_v,to_v,fouls_v,
                                  ts_per_v,threes_ar_v,ft_ar_v,drb_per_v,trb_per_v,ast_per_v,
                                  stl_per_v,blk_per_v,user_per_v,drtg_v,fg_h,fga_h,fg_per_h,
                                  threes_h,threes_att_h,threes_per_h,ft_h,fta_h,ft_per_h,
                                  orb_h,drb_h,trb_h,ast_h,stl_h,blk_h,to_h,fouls_h,ts_per_h,
                                  threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,
                                  blk_per_h,user_per_h,drtg_h]

In [ ]:
len(features)


In [ ]:
#make a dictionary of all the stats pulled from the gamepage
game_dict = dict(zip(headers,[game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,
                                  orb_v,orb_h,ft_fga_v,ft_fga_h,ortg_v,ortg_h,fg_v,fga_v,
                                  fg_per_v,threes_v,threes_att_v,threes_per_v,ft_v,fta_v,
                                  ft_per_v,orb_v,drb_v,trb_v,ast_v,stl_v,blk_v,to_v,fouls_v,
                                  ts_per_v,threes_ar_v,ft_ar_v,drb_per_v,trb_per_v,ast_per_v,
                                  stl_per_v,blk_per_v,user_per_v,drtg_v,fg_h,fga_h,fg_per_h,
                                  threes_h,threes_att_h,threes_per_h,ft_h,fta_h,ft_per_h,
                                  orb_h,drb_h,trb_h,ast_h,stl_h,blk_h,to_h,fouls_h,ts_per_h,
                                  threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,
                                  blk_per_h,user_per_h,drtg_h]))
game_dict

# Scratch Work Graveyard

In [ ]:
'box-score-{}-basic'.format(away)

In [ ]:
soup